# Faiss Vector Store

## Loading Documents and Text Splitter

In [1]:
# import the dataset
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

/home/prasanth/projects/Langchain/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the documents
loader = TextLoader("../data/speech.txt")

docs = loader.load()

In [3]:
# Split the documents
splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

split_docs[:5]

[Document(metadata={'source': '../data/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material'),
 Document(metadata={'source': '../data/speech.txt'}, page_content='to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights'),
 Document(metadata={'source': '../data/speech.txt'}, page_content='We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': '../data/speech.txt'}, page_content='Just because we fight without rancor and without selfish o

## Create Embeddings and Vector Store

In [4]:
embeddings = OllamaEmbeddings(model="mistral:7b")
vector_store =  FAISS.from_documents(split_docs, embeddings)

## Similarity Search

In [7]:
# perform similarity search
query_result = vector_store.similarity_search("What is the consequence of war")
query_result

[Document(id='65adbb83-ddd8-4fa6-9310-ceeae8792b8b', metadata={'source': '../data/speech.txt'}, page_content='we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='8ec67d20-aef1-4133-aa22-1a0bdce717ad', metadata={'source': '../data/speech.txt'}, page_content='…'),
 Document(id='e9ebe404-9e48-4a34-9a1c-f6f217a9cdc5', metadata={'source': '../data/speech.txt'}, page_content='mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'),
 Document(id='b7617108-f527-4d99-9924-cceeec733c41', metadata={'source': '../data/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we 

In [8]:
query_result[0].page_content

'we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

## Similarity Search with Score

In [13]:
vector_store.similarity_search_with_score("What is the consequence of war")

[(Document(id='65adbb83-ddd8-4fa6-9310-ceeae8792b8b', metadata={'source': '../data/speech.txt'}, page_content='we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
  np.float32(1.5426034)),
 (Document(id='8ec67d20-aef1-4133-aa22-1a0bdce717ad', metadata={'source': '../data/speech.txt'}, page_content='…'),
  np.float32(1.6213167)),
 (Document(id='e9ebe404-9e48-4a34-9a1c-f6f217a9cdc5', metadata={'source': '../data/speech.txt'}, page_content='mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'),
  np.float32(1.6549212)),
 (Document(id='b7617108-f527-4d99-9924-cceeec733c41', metadata={'source': '../data/speech.txt'}, page_content='Just because we fight wit

## As a Retriever

We can also convert vector store into a retriever class. This allows us to directly use it in other Langchain methods, which largely work with retrievers.

In [11]:
retriever = vector_store.as_retriever()
query_result = retriever.invoke("What is the consequence of war")
query_result

[Document(id='65adbb83-ddd8-4fa6-9310-ceeae8792b8b', metadata={'source': '../data/speech.txt'}, page_content='we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='8ec67d20-aef1-4133-aa22-1a0bdce717ad', metadata={'source': '../data/speech.txt'}, page_content='…'),
 Document(id='e9ebe404-9e48-4a34-9a1c-f6f217a9cdc5', metadata={'source': '../data/speech.txt'}, page_content='mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'),
 Document(id='b7617108-f527-4d99-9924-cceeec733c41', metadata={'source': '../data/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we 

In [12]:
query_result[0].page_content

'we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

## Save Local

In [14]:
vector_store.save_local("faiss_index")

## Load Local

In [17]:
vector_store.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [18]:
vector_store.similarity_search("What is the consequence of war")

[Document(id='65adbb83-ddd8-4fa6-9310-ceeae8792b8b', metadata={'source': '../data/speech.txt'}, page_content='we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(id='8ec67d20-aef1-4133-aa22-1a0bdce717ad', metadata={'source': '../data/speech.txt'}, page_content='…'),
 Document(id='e9ebe404-9e48-4a34-9a1c-f6f217a9cdc5', metadata={'source': '../data/speech.txt'}, page_content='mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance except from a lawless and malignant few.'),
 Document(id='b7617108-f527-4d99-9924-cceeec733c41', metadata={'source': '../data/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we 